In [1]:
import IPython
import pandas as pd
from scipy.io import wavfile

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.layers import LSTM, Dropout,Dense, BatchNormalization, GRU
from tensorflow.keras import activations
from tensorflow.keras import regularizers
from pickle import dump

from helper import *

## Data Creation Hyperparameters

In [2]:
chunk_size = 500
input_len = 20
output_len = 20
stride = 1

In [ ]:
x_train, y_train, x_test, y_test = preprocess_pipeline(start = 0, end = 10, trainTest = True,
                                                       chunk_size = chunk_size, input_len = input_len,
                                                       output_len = output_len, category='other')

beginning loading data
0
2
1
2
2
2
3
2
4
2
5
2
6
2
7
2
8
2
9
2


## Model Creation

In [ ]:
def create_encoder_decoder(num_frequency_dimensions, num_hidden_dimensions, num_recurrent_units=1, 
                            input_len = 7, output_len = 1):
    model = tf.keras.Sequential()
    
    # encoder
    model.add(tf.keras.layers.InputLayer(input_shape = (input_len, num_frequency_dimensions)))
    model.add(tf.keras.layers.TimeDistributed(Dense(num_hidden_dimensions)))
    for cur_unit in range(num_recurrent_units):
        model.add(tf.keras.layers.LSTM(num_hidden_dimensions, return_sequences=False))
    
    # decoder
    model.add(tf.keras.layers.RepeatVector(input_len))
    model.add(tf.keras.layers.LSTM(2048, activation='relu', return_sequences=True))
    model.add(tf.keras.layers.TimeDistributed(Dense(num_frequency_dimensions)))

    model.summary()
    return model

In [ ]:
model = create_encoder_decoder(2 * chunk_size, 2048, num_recurrent_units = 1,
                                input_len = input_len, output_len = output_len)

## Model Training

In [ ]:
lr = 0.0001
print(lr)
opt = tf.optimizers.Adam(lr=lr)
met = tf.metrics.MeanAbsoluteError()
model.compile(loss=tf.losses.MSE, optimizer=opt, metrics=[met])
model.fit(x_train, y_train, epochs=100, shuffle=True)

## Seeing Results

In [ ]:
pred = model.predict(x_train[0:100])
pred.shape

predTest = model.predict(x_test[0:100])
predTest.shape

In [ ]:
songnum = 1
plot_prediction(x_train[songnum], y_train[songnum], model)

In [ ]:
songnum = 1
pred_song = plot_prediction(x_test[songnum], y_test[songnum], model)

In [ ]:
IPython.display.Audio((pred_song * MAX_VAL).astype(int), rate = 4410)

In [ ]:
songum = 7
curr_song = predict_many(x_test[songum], model, num_predictions = 30, overlap=5):

In [ ]:
IPython.display.Audio(curr_song * MAX_VAL, rate = 4410)

In [ ]:
model.save("saved_models/ae2")